In [1]:
cd '/userdata/pmavrodi/Projekte/Stao/src'

/userdata/pmavrodi/Projekte/Stao/src


In [2]:


import logging
import pandas as pd
import numpy as np
import configparser
import os

from simple_logging.custom_logging import setup_custom_logger
from input_reader.input_reader import get_input

settingsFile = "settings.cfg"

# logger einrichten
LOGGING_LEVEL = logging.INFO
logger = setup_custom_logger('GM_LOGGER', LOGGING_LEVEL, flog="logs/gm.log")

In [3]:
    config = configparser.ConfigParser()
    config.read(settingsFile)
    use_cache = config.getboolean('global', 'cache_enabled')
    cache_dir = config['cache_config']['cache_dir']
    single_store = config['global']['single_store']

    drivetimes_pd = pd.read_pickle(os.path.join(cache_dir,
                                                        config['cache_config']['drivetimes_cached']))

In [4]:
(stores_pd, stores_migros_pd, drivetimes_pd, haushalt_pd) = get_input(settingsFile, logger)

In [5]:
stores_migros_pd

,ID,RETAILER,FORMAT,STRASSE,PLZ,ORT,VERKAUFSFLAECHE_TOTAL,VERKAUFSFLAECHE,Y,X,RELEVANZ,PROFIT_KST,UMSATZ_FOOD,UMSATZ_FRISCHE,UMSATZ_NEAR_NONFOOD,UMSATZ_FACHMARKT,type
OBJECTID,,,,,,,,,,,,,,,,,
834,SM_MIG_68080_24815,Migros,M,Badenerstrasse 376,8004,Zürich,1064.582,516.58,680806.9627,248154.6014,1.0,150327.0,3756359.05,7359424.2,2156160.77,226835.98,MIG


In [9]:
drivetimes_pd=drivetimes_pd.reset_index().set_index(keys='filiale_id')

In [10]:
 relevant_hektars = set(drivetimes_pd.loc[stores_migros_pd.ID]['hektar_id'])

In [11]:
drivetimes_rel_hektars_pd = drivetimes_pd[drivetimes_pd['hektar_id'].isin(relevant_hektars)]

In [12]:
drivetimes_rel_hektars_stores_pd = drivetimes_rel_hektars_pd.merge(
    stores_pd[['ID', 'FORMAT', 'VERKAUFSFLAECHE', 'VERKAUFSFLAECHE_TOTAL', 'RELEVANZ', 'type']],
    left_index=True, right_on='ID', how='inner')

In [13]:
enriched_pd = drivetimes_rel_hektars_stores_pd.merge(haushalt_pd[['H14PTOT']], left_on='hektar_id', right_index=True)

In [14]:
enriched_pd['H14PTOT_corrected'] = enriched_pd['H14PTOT'].fillna(1)

In [15]:
enriched_pd['LAT'] = np.where(enriched_pd.VERKAUFSFLAECHE_TOTAL < 1000,
                                      enriched_pd.RELEVANZ * enriched_pd.VERKAUFSFLAECHE_TOTAL * 0.06,
                                      np.where((enriched_pd.VERKAUFSFLAECHE_TOTAL >= 1000) & (
                                                enriched_pd.VERKAUFSFLAECHE_TOTAL < 2500),
                            enriched_pd.RELEVANZ*(20 * (enriched_pd.VERKAUFSFLAECHE_TOTAL - 1000) / 1500 + 60),
                            enriched_pd.RELEVANZ*(20 * (enriched_pd.VERKAUFSFLAECHE_TOTAL - 2500) / 3500 + 80)))

enriched_pd['RLAT'] = enriched_pd['LAT'] * np.power(10, -(np.fmin(enriched_pd['LAT']/60.0, 1.0)
                                                             *(0.04-0.1)+0.1)*enriched_pd['fahrzeit'])

In [16]:
enriched_pd = enriched_pd.reset_index().set_index(keys=['hektar_id', 'type', 'OBJECTID'])

In [27]:
enriched_pd=enriched_pd[~enriched_pd.index.duplicated(keep='first')]

In [39]:
enriched_pd = enriched_pd.reset_index().set_index(keys=['hektar_id', 'type', 'OBJECTID', 'fahrzeit']).sort_index(level=[0,1,3])

In [43]:
enriched_pd = enriched_pd.reset_index()

In [45]:
enriched_pd.head(50)

,hektar_id,type,OBJECTID,fahrzeit,ID,FORMAT,VERKAUFSFLAECHE,VERKAUFSFLAECHE_TOTAL,RELEVANZ,H14PTOT,H14PTOT_corrected,LAT,RLAT
0,64542616,ALD,2887,4,SM_ALD_64465_26140,Aldi Supermarkt,115.0,1000.0,0.8,3,3,48.000000,29.733172
1,64542616,ALD,2828,18,SM_ALD_64677_24927,Aldi Supermarkt,115.0,1000.0,0.8,3,3,48.000000,5.562131
2,64542616,ALD,2881,18,SM_ALD_66431_25458,Aldi Supermarkt,115.0,1000.0,0.8,3,3,48.000000,5.562131
3,64542616,ALD,2853,19,SM_ALD_64852_24872,Aldi Supermarkt,115.0,1000.0,0.8,3,3,48.000000,4.934478
4,64542616,ALD,2885,21,SM_ALD_62132_26115,Aldi Supermarkt,115.0,1000.0,0.8,3,3,48.000000,3.883660
5,64542616,ALD,2889,21,SM_ALD_66070_26013,Aldi Supermarkt,115.0,1000.0,0.8,3,3,48.000000,3.883660
6,64542616,ALD,2932,21,SM_ALD_65583_24994,Aldi Supermarkt,115.0,1000.0,0.8,3,3,48.000000,3.883660
7,64542616,ALD,2993,21,SM_ALD_66701_25761,Aldi Supermarkt,1320.0,1000.0,0.8,3,3,48.000000,3.883660
8,64542616,ALD,2969,22,SM_ALD_67027_25292,Aldi Supermarkt,115.0,1000.0,0.8,3,3,48.000000,3.445413
9,64542616,ALD,2936,23,SM_ALD_64664_24536,Aldi Supermarkt,115.0,1000.0,0.8,3,3,48.000000,3.056619


In [46]:
def pr(x):
    x['diff1'] = x['RLAT'] - x['RLAT'].iloc[0]
    return x

enriched_pruned_pd = enriched_pd.groupby(by=['hektar_id', 'type']).apply(pr)

# sorted['diff1'] = sorted['RLAT'] - sorted['RLAT'].iloc[0]
# sorted['diff2'] = sorted['RLAT'].iloc[1:len(sorted)] - sorted['RLAT'].shift(1)
# sorted = sorted[(sorted['diff1'] >= 0) & ( (sorted['diff2'] >=0 ) | np.isnan(sorted['diff2']))] 

In [28]:
a=enriched_pd.head()
a
# a.loc[a['fahrzeit']==14,'RLAT']=32
# a.loc[a['fahrzeit']==19,'RLAT']=34

fahrzeit                  ID FORMAT  VERKAUFSFLAECHE  \
hektar_id type OBJECTID                                                         
66322435  MIG  8               10  SM_MIG_66835_23607     MM          1001.19   
               18              19  SM_MIG_64852_24674   SPEZ             0.00   
               26              28  SM_MIG_67548_25665    VOI           220.00   
               39              14  SM_MIG_66250_25169      M           698.26   
               47              26  SM_MIG_67962_24716    VOI           220.00   

                         VERKAUFSFLAECHE_TOTAL  RELEVANZ  H14PTOT  \
hektar_id type OBJECTID                                             
66322435  MIG  8                      1787.864       1.0       10   
               18                      330.895       1.0       10   
               26                      220.000       1.0       10   
               39                     1098.812       1.0       10   
               47                      220.000       1.0       10   

                         H14PTOT_corrected        LAT       RLAT  
hektar_id type OBJECTID                                           
66322435  MIG  8                        10  70.504853  28.068488  
               18                       10  19.853700   0.595747  
               26                       10  13.200000   0.048998  
               39                       10  61.317493  16.888240  
               47                       10  13.200000   0.073076

In [29]:
def prune(x):
    sorted = x.sort_values(axis='index', by='fahrzeit')
    # sorted['diff1'] = sorted['RLAT'] - sorted['RLAT'].iloc[0]
    # sorted['diff2'] = sorted['RLAT'].iloc[1:len(sorted)] - sorted['RLAT'].shift(1)
    # sorted = sorted[(sorted['diff1'] >= 0) & ( (sorted['diff2'] >=0 ) | np.isnan(sorted['diff2']))] 
    return sorted
    
grouped=a.groupby(level=[0,1]).apply(prune)


In [47]:
enriched_pruned_pd.head()

,hektar_id,type,OBJECTID,fahrzeit,ID,FORMAT,VERKAUFSFLAECHE,VERKAUFSFLAECHE_TOTAL,RELEVANZ,H14PTOT,H14PTOT_corrected,LAT,RLAT,diff1
0,64542616,ALD,2887,4,SM_ALD_64465_26140,Aldi Supermarkt,115.0,1000.0,0.8,3,3,48.0,29.733172,0.000000
1,64542616,ALD,2828,18,SM_ALD_64677_24927,Aldi Supermarkt,115.0,1000.0,0.8,3,3,48.0,5.562131,-24.171040
2,64542616,ALD,2881,18,SM_ALD_66431_25458,Aldi Supermarkt,115.0,1000.0,0.8,3,3,48.0,5.562131,-24.171040
3,64542616,ALD,2853,19,SM_ALD_64852_24872,Aldi Supermarkt,115.0,1000.0,0.8,3,3,48.0,4.934478,-24.798693
4,64542616,ALD,2885,21,SM_ALD_62132_26115,Aldi Supermarkt,115.0,1000.0,0.8,3,3,48.0,3.883660,-25.849511


In [34]:
    def calc_MA(x):
        x['Marktanteil'] = x['RLAT'] / np.nansum(x['RLAT'])
        return x


In [35]:
enriched_pruned_pd.groupby(level=2).apply(calc_MA)

fahrzeit                  ID  \
hektar_id type hektar_id type OBJECTID                                 
64742611  DEN  64742611  DEN  1598             7  SM_DEN_65178_25900   
          LAN  64742611  LAN  2389            10  SM_LAN_64567_26555   
          MIG  64742611  MIG  976              5  SM_MIG_64371_26208   
          VOL  64742611  VOL  4962             2  SM_VOL_64669_26132   
64882489  COO  64882489  COO  3426             3  SM_COO_64768_24895   
          MIG  64882489  MIG  1037             2  SM_MIG_64910_24840   
64882490  DEN  64882490  DEN  1070             3  SM_DEN_64911_24841   
          MIG  64882490  MIG  1037             2  SM_MIG_64910_24840   
64882491  ALD  64882491  ALD  2853             2  SM_ALD_64852_24872   
          DEN  64882491  DEN  1074            10  SM_DEN_65575_24884   
          MIG  64882491  MIG  1037             3  SM_MIG_64910_24840   
65632558  DEN  65632558  DEN  1754             4  SM_DEN_65795_25465   
          MIG  65632558  MIG  981              5  SM_MIG_65807_25465   
          VOL  65632558  VOL  5043             2  SM_VOL_65632_25538   
65632559  COO  65632559  COO  3122             6  SM_COO_65493_25343   
          DEN  65632559  DEN  1827            14  SM_DEN_65200_25044   
          MIG  65632559  MIG  981              5  SM_MIG_65807_25465   
          VOL  65632559  VOL  5043             2  SM_VOL_65632_25538   
66022563  COO  66022563  COO  3717             4  SM_COO_65856_25585   
          DEN  66022563  DEN  1754             5  SM_DEN_65795_25465   
          MIG  66022563  MIG  800              3  SM_MIG_66090_25695   
                              970              6  SM_MIG_65834_25854   
                              164              9  SM_MIG_66076_26019   
                              39              10  SM_MIG_66250_25169   
                              610             12  SM_MIG_65574_24872   
                              591             14  SM_MIG_67034_25305   
                              437             16  SM_MIG_67270_25092   
                              538             19  SM_MIG_67610_25024   
                              426             21  SM_MIG_68071_25270   
                              428             23  SM_MIG_68248_24885   
...                                          ...                 ...   
70552267  MIG  70552267  MIG  451             19  SM_MIG_71240_23327   
                              726             22  SM_MIG_68512_23875   
                              423             26  SM_MIG_68189_24768   
70752276  ALD  70752276  ALD  2880             8  SM_ALD_69860_22736   
          DEN  70752276  DEN  1116             2  SM_DEN_70739_22773   
          MIG  70752276  MIG  574              1  SM_MIG_70732_22772   
          SPA  70752276  SPA  4361             3  SM_SPA_70597_22754   
70942281  DEN  70942281  DEN  1116             5  SM_DEN_70739_22773   
          MIG  70942281  MIG  574              4  SM_MIG_70732_22772   
          SPA  70942281  SPA  4498             3  SM_SPA_71047_22755   
70952273  ALD  70952273  ALD  2880            10  SM_ALD_69860_22736   
          DEN  70952273  DEN  1184             6  SM_DEN_71091_22604   
          MIG  70952273  MIG  49               5  SM_MIG_71080_22596   
                              443              8  SM_MIG_70259_22862   
                              442             12  SM_MIG_70464_23182   
                              462             17  SM_MIG_72275_22106   
                              469             18  SM_MIG_69306_23163   
                              501             21  SM_MIG_68396_23949   
                              435             24  SM_MIG_68216_24061   
          SPA  70952273  SPA  4498             3  SM_SPA_71047_22755   
70952274  ALD  70952274  ALD  2880            10  SM_ALD_69860_22736   
          DEN  70952274  DEN  1184             6  SM_DEN_71091_22604   
          MIG  70952274  MIG  49               5  SM_MIG_71080_22596   
        

In [25]:
enriched_pruned_pd

fahrzeit                  ID  \
hektar_id type hektar_id type OBJECTID                                 
64742611  DEN  64742611  DEN  1598             7  SM_DEN_65178_25900   
          LAN  64742611  LAN  2389            10  SM_LAN_64567_26555   
          MIG  64742611  MIG  976              5  SM_MIG_64371_26208   
          VOL  64742611  VOL  4962             2  SM_VOL_64669_26132   
64882489  COO  64882489  COO  3426             3  SM_COO_64768_24895   
          MIG  64882489  MIG  1037             2  SM_MIG_64910_24840   
64882490  DEN  64882490  DEN  1070             3  SM_DEN_64911_24841   
          MIG  64882490  MIG  1037             2  SM_MIG_64910_24840   
64882491  ALD  64882491  ALD  2853             2  SM_ALD_64852_24872   
          DEN  64882491  DEN  1074            10  SM_DEN_65575_24884   
          MIG  64882491  MIG  1037             3  SM_MIG_64910_24840   
65632558  DEN  65632558  DEN  1754             4  SM_DEN_65795_25465   
          MIG  65632558  MIG  981              5  SM_MIG_65807_25465   
          VOL  65632558  VOL  5043             2  SM_VOL_65632_25538   
65632559  COO  65632559  COO  3122             6  SM_COO_65493_25343   
          DEN  65632559  DEN  1827            14  SM_DEN_65200_25044   
          MIG  65632559  MIG  981              5  SM_MIG_65807_25465   
          VOL  65632559  VOL  5043             2  SM_VOL_65632_25538   
66022563  COO  66022563  COO  3717             4  SM_COO_65856_25585   
          DEN  66022563  DEN  1754             5  SM_DEN_65795_25465   
          MIG  66022563  MIG  800              3  SM_MIG_66090_25695   
                              970              6  SM_MIG_65834_25854   
                              164              9  SM_MIG_66076_26019   
                              39              10  SM_MIG_66250_25169   
                              610             12  SM_MIG_65574_24872   
                              591             14  SM_MIG_67034_25305   
                              437             16  SM_MIG_67270_25092   
                              538             19  SM_MIG_67610_25024   
                              426             21  SM_MIG_68071_25270   
                              428             23  SM_MIG_68248_24885   
...                                          ...                 ...   
70552267  MIG  70552267  MIG  451             19  SM_MIG_71240_23327   
                              726             22  SM_MIG_68512_23875   
                              423             26  SM_MIG_68189_24768   
70752276  ALD  70752276  ALD  2880             8  SM_ALD_69860_22736   
          DEN  70752276  DEN  1116             2  SM_DEN_70739_22773   
          MIG  70752276  MIG  574              1  SM_MIG_70732_22772   
          SPA  70752276  SPA  4361             3  SM_SPA_70597_22754   
70942281  DEN  70942281  DEN  1116             5  SM_DEN_70739_22773   
          MIG  70942281  MIG  574              4  SM_MIG_70732_22772   
          SPA  70942281  SPA  4498             3  SM_SPA_71047_22755   
70952273  ALD  70952273  ALD  2880            10  SM_ALD_69860_22736   
          DEN  70952273  DEN  1184             6  SM_DEN_71091_22604   
          MIG  70952273  MIG  49               5  SM_MIG_71080_22596   
                              443              8  SM_MIG_70259_22862   
                              442             12  SM_MIG_70464_23182   
                              462             17  SM_MIG_72275_22106   
                              469             18  SM_MIG_69306_23163   
                              501             21  SM_MIG_68396_23949   
                              435             24  SM_MIG_68216_24061   
          SPA  70952273  SPA  4498             3  SM_SPA_71047_22755   
70952274  ALD  70952274  ALD  2880            10  SM_ALD_69860_22736   
          DEN  70952274  DEN  1184             6  SM_DEN_71091_22604   
          MIG  70952274  MIG  49               5  SM_MIG_71080_22596   
        

# Testing core algo

In [30]:
stores_migros_pd.head()

,ID,RETAILER,FORMAT,STRASSE,PLZ,ORT,VERKAUFSFLAECHE_TOTAL,VERKAUFSFLAECHE,Y,X,RELEVANZ,PROFIT_KST,UMSATZ_FOOD,UMSATZ_FRISCHE,UMSATZ_NEAR_NONFOOD,UMSATZ_FACHMARKT,own_hektar_id
OBJECTID,,,,,,,,,,,,,,,,,
6,SM_MIG_61607_15939,Migros,M,Kanderstegstrasse 31,3714,Frutigen,878.621,519.42,616073.9153,159391.5490,1.0,33370.0,2714265.89,5007897.01,1647566.21,149192.23,61601593
8,SM_MIG_66835_23607,Migros,MM,Kirchfeldstrasse 8,5630,Muri AG,1787.864,1001.19,668350.1017,236074.5430,1.0,33934.0,4973971.80,10746847.03,3218149.87,476688.61,66832360
9,SM_MIG_55810_14347,Migros,M,Rue du Léman 9,1815,Clarens,1026.810,622.98,558105.4795,143471.3736,1.0,63991.0,4432271.32,10699936.16,2424134.69,233238.81,55811434
11,SM_MIG_49525_11909,Migros,M,Route de Peney 2-4,1214,Vernier,1257.324,649.41,495252.6938,119091.9983,1.0,44860.0,3537224.77,7729667.80,1861354.71,233174.78,49521190
13,SM_MIG_63399_12693,Migros,MM,Brückenweg 10,3930,Visp,2230.170,1101.84,633999.7599,126933.4256,1.0,134480.0,6229558.90,11250268.13,5737413.46,781103.84,63391269


In [4]:
(stores_pd, stores_migros_pd, drivetimes_pd, haushalt_pd) = get_input(settingsFile, logger)
relevant_hektars = set(drivetimes_pd.loc[stores_migros_pd.ID]['hektar_id'])
drivetimes_rel_hektars_pd = drivetimes_pd[drivetimes_pd['hektar_id'].isin(relevant_hektars)]

In [5]:
drivetimes_pd.head()

,fahrzeit,hektar_id
filiale_id,,
SM_MIG_61607_15939,21,61341718
SM_MIG_61607_15939,21,61341719
SM_MIG_61607_15939,21,61341722
SM_MIG_61607_15939,22,61341730
SM_MIG_61607_15939,23,61341731


In [6]:
stores_pd.head()


,ID,RETAILER,FORMAT,STRASSE,PLZ,ORT,VERKAUFSFLAECHE_TOTAL,VERKAUFSFLAECHE,Y,X,RELEVANZ,PROFIT_KST,UMSATZ_FOOD,UMSATZ_FRISCHE,UMSATZ_NEAR_NONFOOD,UMSATZ_FACHMARKT,own_hektar_id
OBJECTID,,,,,,,,,,,,,,,,,
6,SM_MIG_61607_15939,Migros,M,Kanderstegstrasse 31,3714,Frutigen,878.621,519.420,616073.9153,159391.5490,1.0,33370.0,2714265.89,5007897.01,1647566.21,149192.23,61601593
8,SM_MIG_66835_23607,Migros,MM,Kirchfeldstrasse 8,5630,Muri AG,1787.864,1001.190,668350.1017,236074.5430,1.0,33934.0,4973971.80,10746847.03,3218149.87,476688.61,66832360
9,SM_MIG_55810_14347,Migros,M,Rue du Léman 9,1815,Clarens,1026.810,622.980,558105.4795,143471.3736,1.0,63991.0,4432271.32,10699936.16,2424134.69,233238.81,55811434
10,SM_MIG_49997_11718,Migros,SPEZ,Rue du Conseil-Général 20,1205,Genève,157.476,157.476,499972.6443,117183.8769,1.0,NaN,NaN,NaN,NaN,NaN,49991171
11,SM_MIG_49525_11909,Migros,M,Route de Peney 2-4,1214,Vernier,1257.324,649.410,495252.6938,119091.9983,1.0,44860.0,3537224.77,7729667.80,1861354.71,233174.78,49521190


In [5]:
drivetimes_rel_hektars_pd.head()

,fahrzeit,hektar_id
filiale_id,,
SM_MIG_61607_15939,21,61341718
SM_MIG_61607_15939,21,61341719
SM_MIG_61607_15939,21,61341722
SM_MIG_61607_15939,22,61341730
SM_MIG_61607_15939,23,61341731


In [7]:
drivetimes_rel_hektars_stores_pd = drivetimes_rel_hektars_pd.merge(
        stores_pd[['ID','FORMAT', 'VERKAUFSFLAECHE', 'VERKAUFSFLAECHE_TOTAL', 'RELEVANZ']], left_index=True,
    right_on='ID', how='inner')

In [10]:
len(drivetimes_rel_hektars_pd)

110246169

In [11]:
len(drivetimes_rel_hektars_stores_pd)

109974613

In [12]:
drivetimes_rel_hektars_stores_pd.head()   

,fahrzeit,hektar_id,ID,FORMAT,VERKAUFSFLAECHE,VERKAUFSFLAECHE_TOTAL,RELEVANZ
OBJECTID,,,,,,,
6,21,61341718,SM_MIG_61607_15939,M,519.42,878.621,1.0
6,21,61341719,SM_MIG_61607_15939,M,519.42,878.621,1.0
6,21,61341722,SM_MIG_61607_15939,M,519.42,878.621,1.0
6,22,61341730,SM_MIG_61607_15939,M,519.42,878.621,1.0
6,23,61341731,SM_MIG_61607_15939,M,519.42,878.621,1.0


In [13]:
enriched_pd = drivetimes_rel_hektars_stores_pd.merge(haushalt_pd[['H14PTOT']], left_on='hektar_id', right_index=True)

In [14]:
enriched_pd['H14PTOT_corrected'] = enriched_pd['H14PTOT'].fillna(1)

In [16]:
enriched_pd.head()

,fahrzeit,hektar_id,ID,FORMAT,VERKAUFSFLAECHE,VERKAUFSFLAECHE_TOTAL,RELEVANZ,H14PTOT,H14PTOT_corrected
OBJECTID,,,,,,,,,
6,21,61341718,SM_MIG_61607_15939,M,519.42,878.621,1.0,1,1
20,28,61341718,SM_MIG_60500_19797,M,755.95,1300.694,1.0,1,1
530,12,61341718,SM_MIG_61274_17700,FM,820.12,1330.134,1.0,1,1
836,25,61341718,SM_MIG_60926_19134,MM,850.50,1879.651,1.0,1,1
949,29,61341718,SM_MIG_60277_19842,M,334.03,494.532,1.0,1,1


In [18]:
migros_only_pd = enriched_pd[enriched_pd['FORMAT'].isin(list(set(stores_migros_pd['FORMAT'])))]

In [23]:
migros_only_pd.groupby('ID').agg( lambda x: len(x))

,fahrzeit,hektar_id,FORMAT,VERKAUFSFLAECHE,VERKAUFSFLAECHE_TOTAL,RELEVANZ,H14PTOT,H14PTOT_corrected
ID,,,,,,,,
SM_MIG_48928_12169,8858,8858,8858,8858.0,8858.0,8858.0,8858,8858
SM_MIG_49525_11909,11407,11407,11407,11407.0,11407.0,11407.0,11407,11407
SM_MIG_49539_12095,11333,11333,11333,11333.0,11333.0,11333.0,11333,11333
SM_MIG_49640_11777,11323,11323,11323,11323.0,11323.0,11323.0,11323,11323
SM_MIG_49743_11609,10982,10982,10982,10982.0,10982.0,10982.0,10982,10982
SM_MIG_49752_12087,12168,12168,12168,12168.0,12168.0,12168.0,12168,12168
SM_MIG_49768_11948,11684,11684,11684,11684.0,11684.0,11684.0,11684,11684
SM_MIG_49781_11376,10585,10585,10585,10585.0,10585.0,10585.0,10585,10585
SM_MIG_49823_12053,11698,11698,11698,11698.0,11698.0,11698.0,11698,11698


In [ ]:
drivetimes_pd = pd.read_csv(drivetimes, sep=',', header=None, names=['filiale_id', 'fahrzeit', 'hektar_id'],
                                    index_col=0, nrows=110299436)

In [ ]:
stores_pd[stores_pd['FORMAT'].isin(['M', 'MM', 'MMM', 'FM'])]

# Einige Daten issues

In [ ]:
# get the input
(stores_pd, stores_migros_pd, drivetimes_pd, drivetimes_migros_pd, haushalt_pd) = get_input(settingsFile, logger)

In [ ]:
drivetimes_pd.head(3)

In [ ]:
print('Number of entries in drivetimes %d: ' % len(drivetimes_pd))

# Duplikate in drivetimes

In [ ]:
drivetimes_pd2 = drivetimes_pd.reset_index()
drivetimes_pd2 = drivetimes_pd2.set_index(['filiale_id', 'fahrzeit', 'hektar_id'])

In [ ]:
drivetimes_pd2.head(3)

In [ ]:
drivetimes_duplicates = drivetimes_pd2[ drivetimes_pd2.index.duplicated()]

In [ ]:
print('Number of duplicates: %d' % len(drivetimes_duplicates))

In [ ]:
drivetimes_duplicates.head(10)

In [ ]:
a=drivetimes_pd.loc['SM_MIG_68294_26394']
a[ (a['fahrzeit']==11) & (a['hektar_id']==67672614) ]

In [ ]:
drivetimes_duplicates.to_csv('/userdata/pmavrodi/Projekte/Stao/output/drivetimes_duplicates.csv')

# Duplikate in drivetimes_migros_pd

In [ ]:
drivetimes_pd3 = drivetimes_migros_pd.reset_index()
drivetimes_pd3 = drivetimes_pd3.set_index(['filiale_id', 'fahrzeit', 'hektar_id'])

In [ ]:
drivetimes_pd3.head(4)

In [ ]:
drivetimes_duplicates2 = drivetimes_pd3[ drivetimes_pd3.index.duplicated()]

In [ ]:
print('Number of duplicates: %d' % len(drivetimes_duplicates2))

In [ ]:
drivetimes_duplicates2.head(3)

# Filiale in drivetimes, aber nicht in stores_sm. Betreffen sind LAT, RLAT, usw

In [ ]:
# get all relevant hektars, i.e. those from which a Migros store is reachable
relevant_hektars = set(drivetimes_migros_pd['hektar_id'])

In [ ]:
drivetimes_rel_hektars_pd = drivetimes_pd[drivetimes_pd['hektar_id'].isin(relevant_hektars)]

In [ ]:
drivetimes_rel_hektars_stores_pd = drivetimes_rel_hektars_pd.join(
        stores_pd[['FORMAT', 'VERKAUFSFLAECHE', 'VERKAUFSFLAECHE_TOTAL', 'RELEVANZ']], how='left')
len(drivetimes_rel_hektars_pd)

In [ ]:
b=drivetimes_rel_hektars_stores_pd[pd.isnull(drivetimes_rel_hektars_stores_pd['VERKAUFSFLAECHE_TOTAL']) &
                                            pd.isnull(drivetimes_rel_hektars_stores_pd['RELEVANZ'])&
                                            pd.isnull(drivetimes_rel_hektars_stores_pd['FORMAT'])].index.unique()

In [ ]:
b

In [ ]:
drivetimes_rel_hektars_stores_pd[~pd.isnull(drivetimes_rel_hektars_stores_pd['VERKAUFSFLAECHE_TOTAL'])].head(2)

In [ ]:
np.savetxt('/userdata/pmavrodi/Projekte/Stao/output/filiale_nur_drivetimes.txt', b,  fmt='%s')

# Hektare von drivetimes_sm ohne HH. Betrifft ist (lokal) Umsatzpotenztal

In [ ]:
enriched_pd = drivetimes_rel_hektars_stores_pd[~pd.isnull(drivetimes_rel_hektars_stores_pd['VERKAUFSFLAECHE_TOTAL'])].join(haushalt_pd['H14PTOT'], on='hektar_id')

In [ ]:
enriched_pd.head(5)

In [ ]:
hektars_ohne_HH_info = enriched_pd[pd.isnull(enriched_pd['H14PTOT'])]['hektar_id'].unique()

In [ ]:
len(hektars_ohne_HH_info)

In [ ]:
np.savetxt('/userdata/pmavrodi/Projekte/Stao/output/hektaren_ohne_HH_info.txt', hektars_ohne_HH_info, fmt='%s')

# DEBUGGING MIT BOJAN

In [6]:
import pickle

In [7]:
enriched = pd.read_pickle('/userdata/pmavrodi/Projekte/Stao/output/enriched_pd_with_duplicates.pkl')

In [12]:
enriched['LAT_NEU'] = np.where( enriched.VERKAUFSFLAECHE_TOTAL < 1000, enriched.RELEVANZ*enriched.VERKAUFSFLAECHE_TOTAL*0.06,
                              np.where((enriched.VERKAUFSFLAECHE_TOTAL >= 1000) & (enriched.VERKAUFSFLAECHE_TOTAL < 2500), 
                                      enriched.RELEVANZ*(20*(enriched.VERKAUFSFLAECHE_TOTAL-1000)/1500 + 60), 
                                      enriched.RELEVANZ*(20*(enriched.VERKAUFSFLAECHE_TOTAL-2500)/3500 + 80)))


In [13]:
enriched.head(10)

,OBJECTID,fahrzeit,hektar_id,ID,FORMAT,VERKAUFSFLAECHE,VERKAUFSFLAECHE_TOTAL,RELEVANZ,H14PTOT,H14PTOT_corrected,...,LAT2,RLAT,RLAT2,Marktanteil,Marktanteil2,LokalUP,LokalUP2,LokalUP_corrected,LokalUP2_corrected,LAT_NEU
0,6,21,61341718,SM_MIG_61607_15939,M,519.42,878.621,1.0,1,1,...,6233.04,20.090134,11.876813,0.000474,0.000428,3.699834,3.339322,3.699834,3.339322,52.717260
1,20,28,61341718,SM_MIG_60500_19797,M,755.95,1300.694,1.0,1,1,...,9071.40,1.920244,1.116026,0.000045,0.000040,0.353635,0.313785,0.353635,0.313785,64.009253
2,530,12,61341718,SM_MIG_61274_17700,FM,820.12,1330.134,1.0,1,1,...,9841.44,1030.567971,635.416736,0.024332,0.022905,189.791171,178.655774,189.791171,178.655774,64.401787
3,836,25,61341718,SM_MIG_60926_19134,MM,850.50,1879.651,1.0,1,1,...,10206.00,8.979630,4.063082,0.000212,0.000146,1.653704,1.142389,1.653704,1.142389,71.728680
4,949,29,61341718,SM_MIG_60277_19842,M,334.03,494.532,1.0,1,1,...,4008.36,0.493601,0.333401,0.000012,0.000012,0.090902,0.093740,0.090902,0.093740,29.671920
5,953,30,61341718,SM_MIG_60110_19901,M,250.90,392.332,1.0,1,1,...,3010.80,0.264749,0.169310,0.000006,0.000006,0.048757,0.047604,0.048757,0.047604,23.539920
6,956,17,61341718,SM_MIG_61580_17713,M,635.94,966.043,1.0,1,1,...,7631.28,105.725002,69.598101,0.002496,0.002509,19.470498,19.568422,19.470498,19.568422,57.962580
7,958,16,61341718,SM_MIG_61034_18031,M,660.20,1163.531,1.0,1,1,...,7922.40,188.347216,106.870235,0.004447,0.003852,34.686347,30.047973,34.686347,30.047973,62.180413
8,963,23,61341718,SM_MIG_63293_16841,M,208.23,335.281,1.0,1,1,...,2498.76,3.504211,2.176329,0.000083,0.000078,0.645341,0.611904,0.645341,0.611904,20.116860
9,966,11,61341718,SM_MIG_61433_17711,M,775.58,1327.339,1.0,1,1,...,9306.96,1521.118685,888.807780,0.035914,0.032038,280.131835,249.900000,280.131835,249.900000,64.364520


In [11]:
20/3500

0.005714285714285714

In [50]:
enriched.loc[ idx, 'LAT_NEU' ] = enriched.loc[idx, 'RELEVANZ'] * (enriched.loc[idx, 'VERKAUFSFLAECHE_TOTAL']/1000.0) * 60

In [10]:
enriched.head(4)

,OBJECTID,fahrzeit,hektar_id,ID,FORMAT,VERKAUFSFLAECHE,VERKAUFSFLAECHE_TOTAL,RELEVANZ,H14PTOT,H14PTOT_corrected,...,LAT2,RLAT,RLAT2,Marktanteil,Marktanteil2,LokalUP,LokalUP2,LokalUP_corrected,LokalUP2_corrected,LAT_NEU
0,6,21,61341718,SM_MIG_61607_15939,M,519.42,878.621,1.0,1,1,...,6233.04,20.090134,11.876813,0.000474,0.000428,3.699834,3.339322,3.699834,3.339322,NaN
1,20,28,61341718,SM_MIG_60500_19797,M,755.95,1300.694,1.0,1,1,...,9071.40,1.920244,1.116026,0.000045,0.000040,0.353635,0.313785,0.353635,0.313785,NaN
2,530,12,61341718,SM_MIG_61274_17700,FM,820.12,1330.134,1.0,1,1,...,9841.44,1030.567971,635.416736,0.024332,0.022905,189.791171,178.655774,189.791171,178.655774,NaN
3,836,25,61341718,SM_MIG_60926_19134,MM,850.50,1879.651,1.0,1,1,...,10206.00,8.979630,4.063082,0.000212,0.000146,1.653704,1.142389,1.653704,1.142389,NaN
